M0 and M1 issues
Change number of components to see the effect
There should only be one eigenvalue, not two
Alpha vectors should only be two values

\begin{align}
H_\alpha\phi(x)\equiv \bigg(-\frac{d^2}{dx^2}+\alpha x^2\bigg)\phi(x)=\lambda\phi(x),\quad \langle\phi|\phi\rangle=1,
\end{align}

\begin{align}
T\phi(x)=-\frac{d^2}{dx^2}\phi(x),\quad V_\alpha(x)\phi(x)=\alpha x^2\phi(x),\quad H_\alpha(x)=T+V(x).
\end{align}


\begin{align}
\bigg(-\frac{d^2}{dx^2}+\alpha x^2-\lambda\bigg)\phi(x)\equiv F_\alpha(\phi(x))=0,
\end{align}


\begin{align}
F_\alpha(\phi(x))=0,\quad\langle\phi|\phi\rangle=1.
\end{align}

\begin{align}
	\hat{\phi}_{\alpha_{k}}(x) = \sum_{i=1}^{n} a_{i} \phi_{i}(x) .
\end{align}

Because we are solving an eigenvalue problem, we can arrive at a set of different-looking equations, using the same set of judges $\{\psi_i(x)\}_{i=1}^n$ as before. We can simply plug $\hat{\phi}_{\alpha_k}$ into the Schrodinger equation and project both sides onto the judges, writing

\begin{align}
\sum_{i=1}^na_i\langle \psi_j|H_{\alpha_k}|\phi_i\rangle=\lambda\sum_{i=1}^na_i\langle\psi_j|\phi_i\rangle.
\end{align}

Define now two matrices:

\begin{align}
M_{ij}(\alpha)\equiv \langle\psi_j|H_\alpha|\phi_i\rangle,\quad N_{ij}\equiv\langle\psi_j|\phi_i\rangle.
\end{align}

These are both $n\times n$ matrices, and we now have a generalized eigenvalue problem for $\vec{a}$:

\begin{align}
M(\alpha)\vec{a}=\lambda N\vec{a}.
\end{align}

Situationally, this may be quicker to solve than finding the roots of the nonlinear system that results from the "traditional" RBM approach.

As with traditional RBM approaches, this is only helpful if we can evaluate $M(\alpha)$ quickly for different $\alpha$ values. For the HO, this is not too hard: we can write

\begin{align}
\langle\psi_j|H_\alpha|\phi_i\rangle=\langle\psi_j|T|\phi_i\rangle+\alpha\langle \psi_j|x^2|\phi_i\rangle\equiv M_{ij}^{(0)}+\alpha M_{ij}^{(1)}.
\end{align}

Our eigenvalue equation is then

\begin{align}
[M^{(0)}+\alpha M^{(1)}]\vec{a}=\lambda N\vec{a},
\end{align}

and all of $M^{(0)},M^{(1)}$ and $N$ can be precomputed.

In [20]:
import numpy as np
import scipy as sci
from scipy import linalg
import scipy
import matplotlib.pyplot as plt
import time as ti

In [32]:
alphas =  [.5,2,5,7,10,15]
x_max = 10.0
h = 10**(-2)
x = np.arange(-x_max, x_max + h, h)
m = np.zeros((len(alphas), x.shape[0]))

In [3]:
# Second Derivative
# Potential Matrix

def second_derivative_matrix(xgrid):
    size = len(xgrid)
    offDiag = np.zeros(size)
    offDiag[1] = 1
        
    H = -1*(-2*np.identity(size) + scipy.linalg.toeplitz(offDiag))/h**2
    return H
        

def potential_matrix(xgrid):
    return np.diag(xgrid**2)

def H_creator(alpha, xgrid):
    d2 = second_derivative_matrix(xgrid)
    pot = potential_matrix(xgrid)
    H = d2 + alpha*pot
    return H

def hf_solve(H):
    evals, evects = np.linalg.eigh(H)
    return evals, evects


In [4]:
for i,alpha in enumerate(alphas):
    alpha = alphas[i]
    H = H_creator(alpha, x)
    evals, evects = hf_solve(H)
    # m[i] = evects[0] / np.linalg.norm(evects[0])*np.sign(evects[0][  int(len(x)/2)  ])
    m[i] = evects[:,0] 

In [5]:
U, sigma, Vh = np.linalg.svd(m)
#components = 3
components = len(m)
print(components)
reduced_basis = Vh[:components]

reduced_basis = [reduced_basis[i]*np.sign(reduced_basis[i][  int(len(x)/2)  ]) for i in range(len(reduced_basis))]

6


In [6]:
# psi = np.array(reduced_basis)
# phi = np.array(reduced_basis)
psi = m
phi = m
print(m)

[[ 4.00124646e-18  8.02148928e-18  1.20828679e-17 ...  1.20833213e-17
   8.02284198e-18  4.00151563e-18]
 [-1.66909538e-21 -2.79628406e-21 -3.05865541e-21 ...  1.27659818e-21
   4.60326011e-22  5.16160702e-22]
 [ 1.49347460e-21  3.21504425e-21  1.37828970e-21 ... -2.19606113e-21
  -1.98874899e-21 -1.09529053e-21]
 [-9.46915006e-22 -1.64402147e-21 -2.23007893e-21 ...  2.24227985e-21
   1.69944257e-21 -7.56249875e-22]
 [-4.23102883e-22  7.03930701e-22  1.92112697e-22 ... -5.50623115e-21
  -3.81475019e-21 -2.54854347e-21]
 [-1.66263303e-22 -1.74493750e-21 -4.40928626e-21 ...  4.18801544e-21
   3.46547318e-21  6.52231913e-22]]


In [31]:
d2 = second_derivative_matrix(x)
pot = potential_matrix(x)

In [8]:
# def M0(psi, phi, d2, i, j):
#     inner_product = np.dot(psi[j], phi[i])
#     result_vector = np.dot(d2, psi[j])
#     M0 = np.dot(result_vector, phi[i])
#     return M0

def M0(psi, phi, d2, i, j):
    M0 = np.dot(psi[j], np.dot(d2, phi[i]))
    return M0

In [9]:
# def M1(psi, phi, pot, i, j):
#     inner_product = np.dot(psi[j], phi[i])
#     result_vector = np.dot(pot, psi[j])
#     M1 = np.dot(result_vector, phi[i])
#     return M1

def M1(psi, phi, pot, i, j):
    M1 = np.dot(psi[j], np.dot(pot, phi[i]))
    return M1

In [10]:
compvec = np.zeros(components)
array = []
for i in range(components):
    array.append(compvec)
H_hat = np.array(array)
M1 = np.array(array)
M2 = np.array(array)
print(len(H_hat))
N = np.array(array)


6


[ ⟨𝜓₀|d2|𝜙₀⟩ + α⟨𝜓₁|pot|𝜙₀⟩   ⟨𝜓₁|d2|𝜙₀⟩ + α⟨𝜓₁|pot|𝜙₀⟩   ⟨𝜓₂|d2|𝜙₀⟩ + α⟨𝜓₂|pot|𝜙₀⟩ ]
[ ⟨𝜓₀|d2|𝜙₁⟩ + α⟨𝜓₀|pot|𝜙₁⟩   ⟨𝜓₁|d2|𝜙₁⟩ + α⟨𝜓₁|pot|𝜙₁⟩   ⟨𝜓₂|d2|𝜙₁⟩ + α⟨𝜓₂|pot|𝜙₁⟩ ]
[ ⟨𝜓₀|d2|𝜙₂⟩ + α⟨𝜓₀|pot|𝜙₂⟩   ⟨𝜓₁|d2|𝜙₂⟩ + α⟨𝜓₁|pot|𝜙₂⟩   ⟨𝜓₂|d2|𝜙₂⟩ + α⟨𝜓₂|pot|𝜙₂⟩ ]

\begin{align}
\sum_{i=1}^na_i\langle \psi_j|H_{\alpha_k}|\phi_i\rangle=\lambda\sum_{i=1}^na_i\langle\psi_j|\phi_i\rangle.
\end{align}

Define now two matrices:

\begin{align}
M_{ij}(\alpha)\equiv \langle\psi_j|H_\alpha|\phi_i\rangle,\quad N_{ij}\equiv\langle\psi_j|\phi_i\rangle.
\end{align}


\begin{align}
\langle\psi_j|H_\alpha|\phi_i\rangle=\langle\psi_j|T|\phi_i\rangle+\alpha\langle \psi_j|x^2|\phi_i\rangle\equiv M_{ij}^{(0)}+\alpha M_{ij}^{(1)}.
\end{align}


In [11]:
def create_H_hat(alpha, phi, psi, pot, d2):
    for i in range(components):
        for j in range(len(H_hat[i])):
            H_hat[i][j] = M0(psi, phi, d2, i, j) + alpha*M1(psi,phi,pot,i,j)
    return H_hat

In [12]:
def create_H_hat(alpha, phi, psi, pot, d2):
    for i in range(components):
        for j in range(i, components):
            M1[i][j] = phi[i] @ d2 @ psi[j]
            M1[j][i] = M1[i][j]
            M2[i][j] = phi[i] @ pot @ psi[j]
            M2[j][i] = M2[i][j]
    H_hat = M1 + alpha*M2
    return H_hat


In [40]:
H_hat = (5, phi, psi, pot, d2)

(5, array([[ 4.00124646e-18,  8.02148928e-18,  1.20828679e-17, ...,
         1.20833213e-17,  8.02284198e-18,  4.00151563e-18],
       [-1.66909538e-21, -2.79628406e-21, -3.05865541e-21, ...,
         1.27659818e-21,  4.60326011e-22,  5.16160702e-22],
       [ 1.49347460e-21,  3.21504425e-21,  1.37828970e-21, ...,
        -2.19606113e-21, -1.98874899e-21, -1.09529053e-21],
       [-9.46915006e-22, -1.64402147e-21, -2.23007893e-21, ...,
         2.24227985e-21,  1.69944257e-21, -7.56249875e-22],
       [-4.23102883e-22,  7.03930701e-22,  1.92112697e-22, ...,
        -5.50623115e-21, -3.81475019e-21, -2.54854347e-21],
       [-1.66263303e-22, -1.74493750e-21, -4.40928626e-21, ...,
         4.18801544e-21,  3.46547318e-21,  6.52231913e-22]]), array([[ 4.00124646e-18,  8.02148928e-18,  1.20828679e-17, ...,
         1.20833213e-17,  8.02284198e-18,  4.00151563e-18],
       [-1.66909538e-21, -2.79628406e-21, -3.05865541e-21, ...,
         1.27659818e-21,  4.60326011e-22,  5.16160702e-22],
  

In [13]:
if phi.all() != psi.all():
    print("???")

In [41]:
def create_N(psi, phi):
    for i in range(components):
        for j in range(i, components):
            N[i,j] = phi[i] @ psi[j]
            N[j,i] = N[i,j]
    return N

In [43]:
N = create_N(psi, phi)
print(N)
# for i in range(len(N)):
#     for j in range(len(N[i])):
#         if N[i][j] < .99999:
#             N[i][j] = 0

[[1.         0.97098301 0.92437601 0.90326415 0.8791515  0.85007729]
 [0.97098301 1.         0.98707835 0.97615658 0.96133942 0.94089519]
 [0.92437601 0.98707835 1.         0.9982346  0.99255765 0.98154524]
 [0.90326415 0.97615658 0.9982346  1.         0.99801672 0.99101853]
 [0.8791515  0.96133942 0.99255765 0.99801672 1.         0.99743872]
 [0.85007729 0.94089519 0.98154524 0.99101853 0.99743872 1.        ]]


In [47]:

def solve(N, H_hat):
    evals, evects = linalg.eigh(H_hat, b=N)
    eigenvalue = evals[0]
    return eigenvalue, evects

eval, a_vecs = solve(N, H_hat)

print(eval)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (5,) + inhomogeneous part.

In [17]:
def ExactEigenvalue(alpha):
    return 2 * (.5) * np.sqrt(alpha / 1**2)
print(ExactEigenvalue(5))

2.23606797749979


In [28]:

def RBM(alpha_k, N):
  H_hat = (alpha, phi, psi, pot, d2)

  time1 = ti.time()
  eval, a_vecs = solve(N, H_hat)
  print(eval)
  time2 = ti.time()
  timeDif = time2-time1
  
  errorDif = ExactEigenvalue(alpha)-eval[0]
  
  return timeDif, errorDif


In [29]:
alphas = [.5, 1, 2, 5, 10, 15]
errors = []
times = []
for alpha in alphas:
  time, error = RBM(alpha, N)
  times.append(time)
  errors.append(error)

maxT = max(times) + 10
minT = min(times) - 10
maxEr = max(errors) + 10
minEr = min(errors) - 10

catFig, catAx = plt.subplots()

# Blue is RBM
catAx.scatter(times,errors,color='blue',
               marker='.',label='RBM')

catAx.set(xscale='log',yscale='log',
           xlabel='Time (s)',ylabel='Eigenvalue Relative Error',
           xlim=(minT, maxT),ylim=(minEr,maxEr))

plt.legend()
plt.show()

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (5,) + inhomogeneous part.